Reading driver json

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [0]:
name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                StructField("surname", StringType(), True)
])

In [0]:
driver_schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                                 StructField("driverRef", StringType(), True),
                                 StructField("number", IntegerType(), True),
                                 StructField("code", StringType(), True),
                                 StructField("name", name_schema),
                                 StructField("dob", DateType(), True),
                                 StructField("nationality", StringType(), True),
                                 StructField("url", StringType(), True)
])

In [0]:
driver_df = spark.read \
.schema(driver_schema) \
.json("abfss://raw@formula1dlusman.dfs.core.windows.net/drivers.json")

Renaming columns and drop unwanted columns

In [0]:
from pyspark.sql.functions import col, concat, current_timestamp, lit

In [0]:
driver_with_columns_df = driver_df \
.withColumnRenamed("driverId", "driver_id") \
.withColumnRenamed("driverRef", "driver_ref") \
.withColumn("ingestion_date", current_timestamp()) \
.withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname"))) \
.drop(col("url"))

##### write the parquet

In [0]:
driver_with_columns_df.write.mode("overwrite").parquet("abfss://processed@formula1dlusman.dfs.core.windows.net/drivers")